In [2]:
import sqlite3
import pandas as pd

connect_one = sqlite3.connect('task1.sqlite')
connect_two = sqlite3.connect('netflix.sqlite')
cursor_request = connect_one.cursor()

movie_sql = """SELECT netflix_titles.show_id AS id, netflix_titles.title AS name, netflix_titles.cast FROM netflix_titles;"""
actor_sql = """SELECT netflix_titles.cast FROM netflix_titles;"""

data_movie = pd.read_sql(movie_sql, connect_two)
data_actor = pd.read_sql(actor_sql, connect_two)

In [3]:
with connect_one:
    cursor_request.execute("""
         CREATE TABLE MOVIE (
             ID INT NOT NULL PRIMARY KEY,
             NAME TEXT,
             CAST TEXT
        );
    """)

In [4]:
with connect_one:
    cursor_request.execute("""
         CREATE TABLE ACTOR (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            NAME TEXT
        );
    """)

In [5]:
with connect_one:
    cursor_request.execute("""
         CREATE TABLE actorANDmovie (
            ACTOR_id INT,
            MOVIE_id INT,
            PRIMARY KEY(ACTOR_id, MOVIE_id),
            FOREIGN KEY(ACTOR_id) REFERENCES ACTOR(id),
            FOREIGN KEY(MOVIE_id) REFERENCES MOVIE(id)
        );
    """)

In [8]:
data_movie.to_sql("MOVIE", con=connect_one, if_exists="append", index=False)

6234

In [9]:
actors = set()


for i in data_actor.values:
    actors_list = i[0].split(', ')
    actors.update(actors_list)

In [10]:
for actor in actors:
    if actor != '':
        cursor_request.execute('INSERT INTO ACTOR (name) VALUES (?)', (actor,))
connect_one.commit()

In [12]:
sql_actorANDmovie = """
    SELECT a.id AS actor_id, m.id AS movie_id 
    FROM ACTOR AS a, MOVIE AS m 
    WHERE m.cast LIKE '% '|| a.name ||' %'
   OR m.cast LIKE '% '||a.name||',%'
   OR m.cast LIKE '% '||a.name
   OR m.cast LIKE a.name||',%';"""
data_actorANDmovie = pd.read_sql(sql_actorANDmovie, connect_one)

In [14]:
data_actorANDmovie.to_sql("actorANDmovie", con=connect_one, if_exists="append", index=False)

44800

In [20]:
sql = """SELECT actor_id1, actor_id2, COUNT(*) AS collaboration_count
FROM (
    SELECT am1.actor_id AS actor_id1, am2.actor_id AS actor_id2
    FROM actorANDmovie am1
    JOIN actorANDmovie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
) AS collaborations
GROUP BY actor_id1, actor_id2
ORDER BY collaboration_count DESC
LIMIT 3;"""

pd.read_sql(sql, connect_one)

,actor_id1,actor_id2,collaboration_count
0,5503,27098,15
1,9109,17473,14
2,9109,20437,14
